### Description:
Pulls Google Anlytics data with pagination and unsampled data

Author:AMARNADH G(INDIA),
Modified by Ameenah Al-Haidari

In [7]:
import pandas as pd

In [33]:

###GOOGLE ANALYICS V4

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import io

todayStr = datetime.today().strftime('%Y-%m-%d')
YstrdyInt = datetime.today() - timedelta(days=1)
YstrdyStr = datetime.strftime(YstrdyInt, '%Y-%m-%d')

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'xxxxxx.json'
VIEW_ID = 'xxxxxx'
PAGESIZE = 100000

In [34]:
def initialize_analyticsreporting():

    credentials = \
        ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION,
            SCOPES)

  # Build the service object.

    analytics = build('analyticsreporting', 'v4',
                      credentials=credentials)

    return analytics

In [35]:
def get_PT(response):
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader',
                {}).get('metricHeaderEntries', [])
        pageToken = report.get('nextPageToken', None)
        print(str(pageToken) + ' at 43')
    return pageToken

In [41]:
def get_report(analytics, pageToken='unknown'):

    return analytics.reports().batchGet(body={'reportRequests': [{
        'viewId': VIEW_ID,
        'pageSize': PAGESIZE,
        'samplingLevel': 'LARGE',
        'pageToken': pageToken,
        'dateRanges': [{'startDate': '2020-01-01',
                       'endDate': '2023-07-24'}],
        'metrics': [{'expression': 'ga:sessions'}],
        'dimensions': [
            {'name': 'ga:clientId'},
            ],
        }]}).execute()

In [42]:
def print_response(response):

    f = io.open('allrowstillOctober' + todayStr + '.csv', 'a+',
                encoding='utf-8')
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader',
                {}).get('metricHeaderEntries', [])

        pageToken=report.get('nextPageToken', None)

    # rint(pageToken)

        print(columnHeader)

      # writing dimention header

        for D_header in dimensionHeaders:
            f.write(str.capitalize(str.replace(D_header, 'ga:', ''))
                    + ',')

        # print(D_header)

        for M_header in list(columnHeader['metricHeader'
                             ]['metricHeaderEntries']):
            f.write(str.capitalize(str.replace(M_header['name'], 'ga:',
                    '')) + ',')

        f.write('\n')

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            Metrics = row.get('metrics', [])

    # writing dimention header row data

            for dimension in dimensions:
                f.write(dimension + ',')

    # writing metric header

            for (i, values) in enumerate(Metrics):
                for (metricHeader, value) in zip(metricHeaders,
                        values.get('values')):
                    f.write(value + ',')
            f.write('\n')
    f.close()


In [45]:
def main():
    analytics = initialize_analyticsreporting()
    response = get_report(analytics)

    pageToken = get_PT(response)

    print(str(pageToken) + ' at 108')

    print_response(response)

    while pageToken:
        print('inside while ' + str(pageToken))
        analytics = initialize_analyticsreporting()
        response = get_report(analytics, pageToken)
        pageToken = get_PT(response)
        print_response(response)
        print(str(pageToken) + ' at 118')


if __name__ == '__main__':
    main()

100000 at 43
100000 at 108
{'dimensions': ['ga:clientId'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}]}}
inside while 100000
200000 at 43
{'dimensions': ['ga:clientId'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}]}}
200000 at 118
inside while 200000
300000 at 43
{'dimensions': ['ga:clientId'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}]}}
300000 at 118
inside while 300000
None at 43
{'dimensions': ['ga:clientId'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}]}}
None at 118


### All Data from 2020-01-01 to 2023-07-24

In [39]:
df = pd.read_csv("allrows2023-10-26.csv")

In [30]:
#df1 = df.drop("Unnamed: 2", axis=1)

In [31]:
df1

,Clientid,Sessions
0,071e2870-78f5-44a4-9086-60aece22914b,1
1,6798a6cd-e45c-443d-ab62-691bb82e7f1b,1
2,e3673c7f-465b-4acf-9872-a211733e2171,1
3,185d195e-d7d0-4479-b562-953b89187acd,1
4,dcd637a5-b3a5-4115-8688-a3acabad40e9,1
...,...,...
303116,999974375.1586809560,2
303117,999983747.1632332319,1
303118,999983974.1685235330,1
303119,999993051.1592417915,1


### Total number of Sessions

In [32]:
pd.to_numeric(df1['Sessions'], errors='coerce').sum()

406864.0